In [2]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
import numpy as np
import matplotlib.pyplot as plt
import random

In [3]:
dataset = pd.read_csv("train.csv", index_col="PassengerId")
print("Initial dataset shape:{}".format(dataset.shape))
#print(dataset.head())
def pipe(dataset):
    
    y = dataset["Survived"]
    dataset = dataset.drop(["Survived"], axis=1)
    x = pretify(dataset, False)
    return (x,y)

ohe = OneHotEncoder()
le = LabelEncoder()

def pretify(dataset, prod=True):
    title_list = ["Mr.", "Master.", "Mrs.", "Miss."]
    
    # http://grammarist.com/usage/mr-mrs-ms-and-miss/
    def map_names(name):
        for title in title_list:
            if title in name:
                return title
        return "Unknown"
            
    dataset["Name"] = dataset["Name"].map(map_names)
    
    print(dataset["Name"].value_counts())
    def slice_and_get_mean_age_by_name(name):
        return dataset.loc[dataset["Name"] == name, "Age"].mean()
    
    age_means = {k:slice_and_get_mean_age_by_name(k) for k in ["Mr.", "Master.", "Mrs.", "Miss.", "Unknown"]}
    
    def fill_age(row):
        if np.isnan(row[3]):
            row[3] = age_means[row[1]]
        return row
        
    dataset = dataset.apply(fill_age, axis=1)
    
    x = dataset.drop(["Cabin", "Embarked","Sex", "Ticket", "Name"], axis=1)
    
    x["Fare"] = x["Fare"].fillna(x["Fare"].mean())
    
    ix=[i for i in dataset.columns if i in ["Embarked", "Sex", "Cabin", "Name"]]
    
    x_cat = dataset[ix];
    
    cabin_list = list("ABCDEFTG")
    def map_cabin(cabin):
        return str(cabin)
    
    x_cat["Cabin"] = x_cat["Cabin"].map(map_cabin)
    
    towns = list("CQS")
    x_cat["Embarked"] = x_cat["Embarked"].fillna(random.choice(towns))
    
    x_cat = x_cat.apply(le.fit_transform)
    
#     if not prod:
#         x_cat.apply((lambda col: le.fit(col)))
        
#     x_cat = x_cat.apply(le.transform)
    
    if not prod:
        ohe.fit(x_cat)
    
    x_cat_encoded = ohe.transform(x_cat)
    x = np.column_stack((x, x_cat_encoded.toarray()))
    
    
    return x

dataset_X, dataset_y = pipe(dataset)
print(dataset_X.shape)

Initial dataset shape:(891, 11)
Mr.        517
Miss.      182
Mrs.       125
Master.     40
Unknown     27
Name: Name, dtype: int64
(891, 163)


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:54: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [4]:
from sklearn.model_selection import train_test_split
train_X , test_X, train_y, test_y = train_test_split(dataset_X, dataset_y, random_state=26)

print(train_X.shape)


(668, 163)


In [5]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression()
model.fit(train_X, train_y)

def print_score(train, test):
    print("Train score is {}".format(train))
    print("Test score is {}".format(test))
    print("")

print_score(model.score(train_X, train_y), model.score(test_X, test_y))
alphas = [0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000]
#alphas = [100]
for alpha in alphas:
    
    from sklearn.linear_model import RidgeClassifier
    model2 = RidgeClassifier(alpha)
    model2.fit(train_X, train_y)
    print("using alpha = {}".format(alpha))
    print("Train score is {}".format(model2.score(train_X, train_y)))
    print("Test score is {}".format(model2.score(test_X, test_y)))
    print("")

Train score is 0.8562874251497006
Test score is 0.8340807174887892

using alpha = 0.0001
Train score is 0.8877245508982036
Test score is 0.8251121076233184

using alpha = 0.001
Train score is 0.8877245508982036
Test score is 0.8251121076233184

using alpha = 0.01
Train score is 0.8877245508982036
Test score is 0.8251121076233184

using alpha = 0.1
Train score is 0.8877245508982036
Test score is 0.8251121076233184

using alpha = 1
Train score is 0.8862275449101796
Test score is 0.8251121076233184

using alpha = 10
Train score is 0.8502994011976048
Test score is 0.8295964125560538

using alpha = 100
Train score is 0.8218562874251497
Test score is 0.8161434977578476

using alpha = 1000
Train score is 0.7619760479041916
Test score is 0.7309417040358744



In [6]:
def save(prediction):
    df = pd.DataFrame(data={"Survived": prediction}, index=test_data["PassengerId"])
    df.to_csv("with_data_cleaning.csv")

In [7]:
from sklearn.neighbors import KNeighborsClassifier

knn_model = KNeighborsClassifier()
knn_model.fit(train_X, train_y)

print_score(knn_model.score(train_X, train_y), knn_model.score(test_X, test_y))

Train score is 0.812874251497006
Test score is 0.7085201793721974



In [8]:
from sklearn.linear_model import SGDClassifier

sdg_model = SGDClassifier()
sdg_model.fit(train_X, train_y)

print_score(knn_model.score(train_X, train_y), knn_model.score(test_X, test_y))

Train score is 0.812874251497006
Test score is 0.7085201793721974



/usr/local/lib/python3.5/dist-packages/sklearn/linear_model/stochastic_gradient.py:84: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


In [9]:
from sklearn.ensemble import RandomForestClassifier

random_f_model = RandomForestClassifier(random_state=0)
random_f_model.fit(train_X, train_y)

print_score(random_f_model.score(train_X, train_y), random_f_model.score(test_X, test_y))

Train score is 0.9730538922155688
Test score is 0.8251121076233184



In [273]:
test_data = pd.read_csv("test.csv")
test_data_processed = pretify(test_data.drop(["PassengerId"], axis=1))

#prediction = random_f_model.predict(test_data_processed)
#prediction = model2.predict(test_data_processed)
prediction = model.predict(test_data_processed)


save(prediction)

Mr.        240
Miss.       78
Mrs.        72
Master.     21
Unknown      7
Name: Name, dtype: int64


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:54: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
